In [1]:
import graphlab
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import scale
from sklearn.linear_model import SGDClassifier
import gensim

C:\Users\OmarAbdelwahab\AppData\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
products = graphlab.SFrame('amazon_baby.gl')

This non-commercial license of GraphLab Create for academic use is assigned to oaabde01@louisville.edu and will expire on September 22, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\OMARAB~1\AppData\Local\Temp\graphlab_server_1483659100.log.0


In [4]:
import os
import gensim
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

BASE_DIR = "/home/graphlab_create/data/blogs" # NOTE: Update BASE_DIR to your own directory path
class TrainSentences(object):
    """
    Iterator class that returns Sentences from texts files in a input directory
    """
    RE_WIHTE_SPACES = re.compile("\s+")
    STOP_WORDS = set(stopwords.words("english"))
    def __init__(self, dirname):
        """
        Initialize a TrainSentences object with a input directory that contains text files for training
        :param dirname: directory name which contains the text files        
        """
        self.dirname = dirname

    def __iter__(self):
        """
        Sentences iterator that return sentences parsed from files in the input directory.
        Each sentences is returned as list of words
        """
        #First iterate  on all files in the input directory
        for fname in os.listdir(self.dirname):
            # read line from file (Without reading the entire file)
            for line in file(os.path.join(self.dirname, fname), "rb"):
                # split the read line into sentences using NLTK
                for s in txt2sentences(line, is_html=True):
                    # split the sentence into words using regex
                    w =txt2words(s, lower=True, is_html=False, remove_stop_words=False,
                                                 remove_none_english_chars=True)
                    #skip short sentneces with less than 3 words
                    if len(w) < 3:
                        continue
                    yield w

def txt2sentences(txt, is_html=False, remove_none_english_chars=True):
    """
    Split the English text into sentences using NLTK
    :param txt: input text.
    :param is_html: If True thenremove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :return: string in which each line consists of single sentence from the original input text.
    :rtype: str
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # split text into sentences using nltk packages
    for s in tokenizer.tokenize(txt):
        if remove_none_english_chars:
            #remove none English chars
            s = re.sub("[^a-zA-Z]", " ", s)
        yield s
    
def txt2words(txt, lower=True, is_html=False, remove_none_english_chars=True, remove_stop_words=True):
    """
    Split text into words list
    :param txt: the input text
    :param lower: if to make the  text to lowercase or not.
    :param is_html: If True then  remove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :param remove_stop_words: if True then remove stop words from text
    :return: words list create from the input text according to the input parameters.
    :rtype: list
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    if lower:
        txt = txt.lower()
    if remove_none_english_chars:
        txt = re.sub("[^a-zA-Z]", " ", txt)

    words = TrainSentences.RE_WIHTE_SPACES.split(txt.strip().lower())
    if remove_stop_words:
        #remove stop words from text
        words = [w for w in words if w not in TrainSentences.STOP_WORDS]
    return words

In [5]:
from numpy import average
import graphlab as gl
import numpy as np
import gensim

class DeepTextAnalyzer(object):
    def __init__(self, word2vec_model):
        """
        Construct a DeepTextAnalyzer using the input Word2Vec model
        :param word2vec_model: a trained Word2Vec model
        """
        self._model = word2vec_model

    def txt2vectors(self,txt, is_html):
        """
        Convert input text into an iterator that returns the corresponding vector representation of each
        word in the text, if it exists in the Word2Vec model
        :param txt: input text
        :param is_html: if True, then extract the text from the input HTML
        :return: iterator of vectors created from the words in the text using the Word2Vec model.
        """
        words = txt2words(txt,is_html=is_html, lower=True, remove_none_english_chars=True)
        words = [w for w in words if w in self._model]
        #yield self._model[txt]
        if len(words) != 0:
            for w in words:
                yield self._model[w]


    def txt2avg_vector(self, txt, is_html):
        """
        Calculate the average vector representation of the input text
        :param txt: input text
        :param is_html: is the text is a HTML
        :return the average vector of the vector representations of the words in the text  
        """
        ##vector = self.txt2vectors(txt,is_html=is_html)
        ##return vector
        vectors = self.txt2vectors(txt,is_html=is_html)
        vectors_sum = next(vectors, None)
        if vectors_sum is None:
            return None
        count =1.0
        for v in vectors:
            count += 1
            vectors_sum = np.add(vectors_sum,v)
        
        #calculate the average vector and replace +infy and -inf with numeric values 
        avg_vector = np.nan_to_num(vectors_sum/count)
        return avg_vector

In [6]:
import graphlab
import os
import gensim
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
def split_sentence(sentence):
    return re.split('\W+', sentence)

In [2]:
tweets = graphlab.SFrame('full_train_2016.tsv')

This non-commercial license of GraphLab Create for academic use is assigned to oaabde01@louisville.edu and will expire on September 22, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\OMARAB~1\AppData\Local\Temp\graphlab_server_1484259696.log.0


Finished parsing file C:\Users\OmarAbdelwahab\full_train_2016.tsv

Parsing completed. Parsed 100 lines in 0.067003 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\OmarAbdelwahab\full_train_2016.tsv

Parsing completed. Parsed 11798 lines in 0.035 secs.

In [8]:
imdb_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/word2vecmodel')

In [9]:
imdb_w2v.most_similar("lol")

[('lol.', 0.5225681066513062),
 (':)', 0.4609519839286804),
 (':).', 0.43749451637268066),
 (';-)', 0.4341832399368286),
 ('alot,', 0.4135943651199341),
 ('no.', 0.4129267930984497),
 (';)', 0.41199108958244324),
 ('poops,', 0.41076964139938354),
 ('haha', 0.40914398431777954),
 ('deserv', 0.4071287512779236)]

In [10]:
tweets['1gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 1)
tweets['2gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 2)
tweets['3gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 3)

In [ ]:
dt = DeepTextAnalyzer(imdb_w2v)
tweets['vectors'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

RuntimeError: Runtime Exception. Runtime Exception: 110. Fail executing the lambda function. The lambda worker may have run out of memory or crashed because it captured objects that cannot be properly serialized.

In [19]:
tweets.head()

TweetID,UserID,Sentiment,Tweet,1gram features
264183816548130816,15140428,positive,ga by my hous hit$NUM.$NUM i m go to ...,"{'on': 1L, 'hit': 1L,'hous': 1L, 'i': 1L, ..."
264249301910310912,18516728,negative,iranian gener say israels iron dome can t deal ...,"{'and': 1L, 'deal': 1L,'we': 1L, 'say': 1L, ..."
264105751826538497,147088367,positive,with j davlar $NUMth.main rival are team ...,"{'week': 1L, 'it': 1L,'an': 1L, 'are': 1L, ..."
264094586689953794,332474633,negative,"talk about act s amp;amp; sat s, decid i ...","{'me': 1L, 'and': 1L,'about': 2L, 'decid': ..."
254941790757601280,557103111,negative,"they may have a superbowlin dalla, but dalla a ...","{'and': 1L, 'a': 2L,'dalla': 2L, 'superbo ..."
264169034155696130,382403760,neutral,"im bring the ne load ofcandi tomorrow, i just ...","{'load': 1L, 'all': 1L,'just': 1L, 'get': 1L, ..."
263192091700654080,344222239,neutral,"appl software, retailchief out in overhaul: ...","{'on': 1L, 'francisco':1L, 'heads': 1L, 'san': ..."
263398998675693568,812957996,positive,AT USER AT USER AT USER ijust watch it sridevi s ...,"{'on': 1L, 'u': 1L,'from': 1L, 'just': 1L, ..."
260200142420992000,332530284,neutral,livewir nadal confirm formexican open in febru ...,"{'me': 1L, 'play': 1L,'nadal': 2L, 'mexican': ..."
264087629237202944,61903760,positive,AT USER i didnt want tojust pop up... but ye ...,"{'and': 1L, 'have': 1L,'just': 1L, 'pop': 1L, ..."


In [8]:
tweets['word_count'] = graphlab.text_analytics.count_words(tweets['Tweet'])

In [9]:
tfidf = graphlab.text_analytics.tf_idf(tweets['word_count'])
tweets['tfidf'] = tfidf

In [99]:
#ignore all neutral tweets
tweets_pos_neg = tweets[tweets['Sentiment'] != 'neutral']
tweets_neutral = tweets[tweets['Sentiment'] == 'neutral']
train_data_pos_neg,test_data_pos_neg = tweets_pos_neg.random_split(.8, seed=0)
feature_set1 = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neg','vectors_pos_neutral']

In [149]:
sentiment_model_word2vec_ =graphlab.logistic_classifier.create(train_data_pos_neg,
                                                     target='Sentiment',
                                                     features=feature_set1,
                                                     max_iterations=100,
                                                     l1_penalty=10,
                                                     l2_penalty=1000.0,
                                                     class_weights = 'auto',
                                                     validation_set=test_data_pos_neg)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2559
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 6
PROGRESS: Number of unpacked features : 87428
PROGRESS: Number of coefficients    : 87429
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000059  | 1.232822     | 0.933568          | 0.788310            |
PROGRESS: | 2         | 3        | 0.000059  | 1.360907     | 

In [5]:
pos_neg_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/pos_neg_model_w2v')
pos_neutral_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/pos_neutral_model_w2v')
neutral_neg_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/neutral_neg_model_w2v')
positive_nonpositive_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/positive_nonpositive_model_w2v')
negative_nonnegative_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/negative_nonnegative_model_w2v')
neutral_nonneutral_model_w2v = Word2Vec.load('C:/Users/OmarAbdelwahab/Documents/RESEARCH/SEMEVAL2016/TwitterSentimentAnalysisTask/Train-Trial-Data/2016Data/Train+dev-2016/Needed/Train/neutral_nonneutral_model_w2v')
doc2vec_reviewsonly_dm = Doc2Vec.load('C:/Python27/doc2vec_dm_reviewsonly')
doc2vec_tweetsonly_dm = Doc2Vec.load('C:/Python27/doc2vec_dm_tweetsonly')

In [ ]:
dt = DeepTextAnalyzer(pos_neg_model_w2v)
tweets['vectors_pos_neg'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

In [42]:
dt = DeepTextAnalyzer(pos_neutral_model_w2v)
tweets['vectors_pos_neutral'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

In [43]:
dt = DeepTextAnalyzer(neutral_neg_model_w2v)
tweets['vectors_neutral_neg'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

In [ ]:
#for loop for looping over the k segments for the K-Fold cross validation
def k_fold_cross_validation(k, l2_penalty, data, output):
    n = len(data)
    k = 10 # 10-fold cross-validation
    RSSTotal = 0
    Accuracy = []
    fscores = []
    
    for i in range(k):
        start = (n*i)/k
        end = (n*(i+1))/k
        Valid_shuffled = data[start:end+1]
        train_shuffled = data[0:start].append(data[end+1:n])
        feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot']
        model =graphlab.logistic_classifier.create(train_shuffled,
                                                     target='NegativeorNot',
                                                     features=feature_set_negative_ornot,
                                                     max_iterations=100,
                                                     l1_penalty=0,
                                                     l2_penalty=l2_penalty,
                                                     class_weights = 'auto',
                                                     validation_set=None)
        modelresults = model.evaluate(Valid_shuffled)
        fscores.append(modelresults['f1_score'])
        Accuracy.append(modelresults['accuracy'])
        ##predictiondifference_squared = np.multiply(np.subtract(modelpredictions,Valid_shuffled['Sentiment']),np.subtract(modelpredictions,Valid_shuffled['Sentiment']))
        ##RSS = predictiondifference_squared.sum()
        ##RSSTotal = RSSTotal + RSS
        ##print modelpredictions
    
    ##RSSAvg = RSSTotal/k
    return (sum(Accuracy)/len(Accuracy),sum(fscores)/len(fscores))
    #return max(fscores)


In [169]:
penalty = []
acc = []
fsco = []
for i in np.linspace(10, 100, num=10):
    (maxAccuracy,maxfscores) = k_fold_cross_validation(10, i, train_data_pos_neg, 'Sentiment')
    penalty.append(i)
    acc.append(maxAccuracy)
    fsco.append(maxfscores)
    #print i

print acc.index(max(acc))
print max(acc)
print fsco.index(max(fsco))
print max(fsco)
print penalty[acc.index(max(acc))]
print penalty[fsco.index(max(fsco))]
    

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2303
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 6
PROGRESS: Number of unpacked features : 79948
PROGRESS: Number of coefficients    : 79949
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000301  | 0.941629     | 0.783326          |
PROGRESS: | 2         | 3        | 0.000301  | 1.052701     | 0.741207          |
PROGRESS: | 3         | 4        | 0.000301  | 1.172782     | 0.7755

In [364]:
penalty = []
acc = []
fsco = []
for i in np.linspace(1000, 10000, num=10):
    (maxAccuracy,maxfscores) = k_fold_cross_validation(10, i, train_data_positive_ornot, 'Sentiment')
    penalty.append(i)
    acc.append(maxAccuracy)
    fsco.append(maxfscores)
    #print i

print acc.index(max(acc))
print max(acc)
print fsco.index(max(fsco))
print max(fsco)
print penalty[acc.index(max(acc))]
print penalty[fsco.index(max(fsco))]
print "l2 penalty"

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8548
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 5
PROGRESS: Number of unpacked features : 244879
PROGRESS: Number of coefficients    : 244880
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | 1         | 3        | 0.000117  | 0.377255     | 0.990407          |
PROGRESS: | 2         | 5        | 1.000000  | 0.676453     | 0.998947          |
PROGRESS: | 3         | 6        | 1.000000  | 0.865578     | 0.867337          |
PROGRESS: | 4         | 8        | 1.000000  | 1.182793     | 0.999181          |
PROGRESS: | 5 

In [102]:
for i in np.logspace(3, 9, num=13):
    RSS = k_fold_cross_validation(10,i,train_data_pos_neg, 'Sentiment')
    print RSS
    print i

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2303
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 6
PROGRESS: Number of unpacked features : 79948
PROGRESS: Number of coefficients    : 79949
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | 1         | 4        | 0.000217  | 0.175117     | 0.992618          |
PROGRESS: | 2         | 6        | 1.000000  | 0.301203     | 0.997829          |
PROGRESS: | 3         | 9        | 0.500000  | 0.482322     | 0.992184          |
PROGRESS: | 4         | 12       | 0.500000  | 0.645430     | 0.999566          |
PROGRESS: | 5   

## Building and tuning the classifiers.

# Let's start with the p/n classifier

In [3]:
tweets = graphlab.SFrame('full_train_2016.tsv')
tweets_doc2vec = graphlab.SFrame('full_train_2016doc2vec.tsv')

tweets['1gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 1)
tweets['2gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 2)
tweets['3gram features'] = gl.text_analytics.count_ngrams(tweets['Tweet'], 3)
tweets.head()

NameError: name 'graphlab' is not defined

In [8]:
dt = DeepTextAnalyzer(pos_neg_model_w2v)
tweets['vectors_pos_neg'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(pos_neutral_model_w2v)
tweets['vectors_pos_neutral'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(neutral_neg_model_w2v)
tweets['vectors_neutral_neg'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
#for the one label classifiers
dt = DeepTextAnalyzer(positive_nonpositive_model_w2v)
tweets['vectors_pos_ornot'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(negative_nonnegative_model_w2v)
tweets['vectors_neg_ornot'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(neutral_nonneutral_model_w2v)
tweets['vectors_neutral_ornot'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

In [22]:
tweets['word_count'] = graphlab.text_analytics.count_words(tweets['Tweet'])
tfidf = graphlab.text_analytics.tf_idf(tweets['word_count'])
tweets['tfidf'] = tfidf
tweets.head()
tweets_doc2vec[1000]

{'[  6.41991734e-04   6.84577913e-04   1.21212285e-03  -1.03691558e-03': '-4.22232561e-02   3.59229334e-02   4.11115922e-02   3.14613571e-03'}

##Will start building and tunning the Pos/Neg classifier

In [2]:
tweets_pos_neg = tweets[tweets['Sentiment'] != 'neutral']
tweets_neutral = tweets[tweets['Sentiment'] == 'neutral']
train_data_pos_neg,test_data_pos_neg = tweets_pos_neg.random_split(.8, seed=0)
tweets_pos_neutral = tweets[tweets['Sentiment'] != 'negative']
train_data_pos_neutral,test_data_pos_neutral = tweets_pos_neutral.random_split(.8, seed=0)
tweets_neutral_neg = tweets[tweets['Sentiment'] != 'positive']
train_data_neutral_neg,test_data_neutral_neg = tweets_neutral_neg.random_split(.8, seed=0)
##dt = DeepTextAnalyzer(doc2vec_reviewsonly_dm)
##tweets['vectors_doc2vec_reviewsonly_dm'] = tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
##'split_sentence' 'split_sentence' 
tweets['vectors_doc2vec_reviewsonly_dm'] = tweets['Tweet'].apply(lambda p: doc2vec_reviewsonly_dm.infer_vector(split_sentence(p)))
##tweets['vectors_doc2vec_reviewsonly_dm'] = tweets_doc2vec.head
##tweets = tweets.append(tweets_doc2vec)

NameError: name 'tweets' is not defined

In [24]:
#feature_set_pos_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neg','vectors_pos_neutral','vectors_pos_ornot','vectors_neg_ornot']
#model_pos_neg =graphlab.logistic_classifier.create(train_data_pos_neg,
                                                     #target='Sentiment',
                                                     #features=feature_set_pos_neg,
                                                     #max_iterations=100,
                                                     #l1_penalty=0,
                                                     #l2_penalty=1000.0,
                                                     #class_weights = 'auto',
                                                     #validation_set=test_data_pos_neg)
tweets.head()

TweetID,Sentiment,Tweet,1gram features,2gram features
264183816548130000,positive,ga by my hous hit$NUM.$NUM i m go to ...,"{'on': 1L, 'hit': 1L,'hous': 1L, 'i': 1L, ...","{'to chapel': 1L, 'numnum': 1L, 'chapel hill': ..."
264249301910310000,negative,iranian gener say israels iron dome can t deal ...,"{'and': 1L, 'deal': 1L,'we': 1L, 'say': 1L, ...","{'with their': 1L, 'domecan': 1L, 'talk po': 1L, ..."
264105751826538000,positive,with j davlar $NUMth.main rival are team ...,"{'week': 1L, 'it': 1L,'an': 1L, 'are': 1L, ...","{'main rival': 1L, 'jdavlar': 1L, 'end to': ..."
264094586689953000,negative,"talk about act s amp;amp; sat s, decid i ...","{'me': 1L, 'and': 1L,'about': 2L, 'decid': ...","{'act s': 1L, 'everythabout': 1L, 'to go': 1L, ..."
254941790757601000,negative,"they may have a superbowlin dalla, but dalla a ...","{'and': 1L, 'a': 2L,'dalla': 2L, 'superbo ...","{'ain t': 1L, 'at user':2L, 'have a': 1L, 'dalla ..."
264169034155696000,neutral,"im bring the ne load ofcandi tomorrow, i just ...","{'load': 1L, 'all': 1L,'just': 1L, 'get': 1L, ...","{'load of': 1L, 'neload': 1L, 'it doesn': ..."
263192091700654000,neutral,"appl software, retailchief out in overhaul: ...","{'on': 1L, 'francisco':1L, 'heads': 1L, 'san': ...","{'inc ceo': 1L, 'overhaulsan': 1L, 'monday ..."
263398998675693000,positive,AT USER AT USER AT USER ijust watch it sridevi s ...,"{'on': 1L, 'u': 1L,'from': 1L, 'just': 1L, ...","{'nums sun': 1L, 'sunmorn': 1L, 'at user': ..."
260200142420992000,neutral,livewir nadal confirm formexican open in febru ...,"{'me': 1L, 'play': 1L,'nadal': 2L, 'mexican': ...","{'mexican open': 1L,'nadal is': 1L, 'febr ..."
264087629237202000,positive,AT USER i didnt want tojust pop up... but ye ...,"{'and': 1L, 'have': 1L,'just': 1L, 'pop': 1L, ...","{'tell her': 1L, 'tojust': 1L, 'just pop': ..."


In [78]:
#feature_set_pos_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neutral','vectors_pos_ornot','vectors_neutral_ornot']
#feature_set_pos_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neutral','vectors_pos_ornot','vectors_neutral_ornot','vectors_doc2vec_tweetsonly_dm']
#feature_set_pos_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_doc2vec_tweetsonly_dm']
feature_set_pos_neutral = ['tfidf','1gram features','2gram features','3gram features']

model_pos_neutral =graphlab.classifier.create(train_data_pos_neutral,
                                                     target='Sentiment',
                                                     features=feature_set_pos_neutral)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 7744
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 221952
PROGRESS: Number of coefficients    : 221953
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [76]:
#feature_set_neutral_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_neg','vectors_neg_ornot','vectors_neutral_ornot']
#feature_set_neutral_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_neg','vectors_neg_ornot','vectors_neutral_ornot','vectors_doc2vec_tweetsonly_dm']
feature_set_neutral_neg = ['tfidf','1gram features','2gram features','3gram features']
#feature_set_neutral_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_doc2vec_tweetsonly_dm']
model_neutral_neg =graphlab.classifier.create(train_data_neutral_neg,
                                                     target='Sentiment',
                                                     features=feature_set_neutral_neg)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5162
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 163307
PROGRESS: Number of coefficients    : 163308
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [26]:
tweets['PositiveorNot'] = tweets['Sentiment'].apply(lambda p: p=='positive')

In [29]:
tweets.head()

TweetID,Sentiment,Tweet,1gram features,2gram features
264183816548130000,positive,ga by my hous hit$NUM.$NUM i m go to ...,"{'on': 1L, 'hit': 1L,'hous': 1L, 'i': 1L, ...","{'to chapel': 1L, 'numnum': 1L, 'chapel hill': ..."
264249301910310000,negative,iranian gener say israels iron dome can t deal ...,"{'and': 1L, 'deal': 1L,'we': 1L, 'say': 1L, ...","{'with their': 1L, 'domecan': 1L, 'talk po': 1L, ..."
264105751826538000,positive,with j davlar $NUMth.main rival are team ...,"{'week': 1L, 'it': 1L,'an': 1L, 'are': 1L, ...","{'main rival': 1L, 'jdavlar': 1L, 'end to': ..."
264094586689953000,negative,"talk about act s amp;amp; sat s, decid i ...","{'me': 1L, 'and': 1L,'about': 2L, 'decid': ...","{'act s': 1L, 'everythabout': 1L, 'to go': 1L, ..."
254941790757601000,negative,"they may have a superbowlin dalla, but dalla a ...","{'and': 1L, 'a': 2L,'dalla': 2L, 'superbo ...","{'ain t': 1L, 'at user':2L, 'have a': 1L, 'dalla ..."
264169034155696000,neutral,"im bring the ne load ofcandi tomorrow, i just ...","{'load': 1L, 'all': 1L,'just': 1L, 'get': 1L, ...","{'load of': 1L, 'neload': 1L, 'it doesn': ..."
263192091700654000,neutral,"appl software, retailchief out in overhaul: ...","{'on': 1L, 'francisco':1L, 'heads': 1L, 'san': ...","{'inc ceo': 1L, 'overhaulsan': 1L, 'monday ..."
263398998675693000,positive,AT USER AT USER AT USER ijust watch it sridevi s ...,"{'on': 1L, 'u': 1L,'from': 1L, 'just': 1L, ...","{'nums sun': 1L, 'sunmorn': 1L, 'at user': ..."
260200142420992000,neutral,livewir nadal confirm formexican open in febru ...,"{'me': 1L, 'play': 1L,'nadal': 2L, 'mexican': ...","{'mexican open': 1L,'nadal is': 1L, 'febr ..."
264087629237202000,positive,AT USER i didnt want tojust pop up... but ye ...,"{'and': 1L, 'have': 1L,'just': 1L, 'pop': 1L, ...","{'tell her': 1L, 'tojust': 1L, 'just pop': ..."


In [29]:
tweets['NegativeorNot'] = tweets['Sentiment'].apply(lambda p: p=='negative')


In [59]:
train_data_positive_ornot,test_data_positive_ornot = tweets.random_split(.8, seed=0)

In [139]:
feature_set_positive_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neutral','vectors_neg_ornot']
model_positive_ornot =graphlab.classifier.create(train_data_positive_ornot,
                                                     target='PositiveorNot',
                                                     features=feature_set_positive_ornot)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9022
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 6
PROGRESS: Number of unpacked features : 256956
PROGRESS: Number of coefficients    : 256957
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [133]:
feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot']
model_negative_ornot =graphlab.logistic_classifier.create(train_data_positive_ornot,
                                                     target='NegativeorNot',
                                                     features=feature_set_negative_ornot,
                                                     max_iterations=100,
                                                     l1_penalty=0,
                                                     l2_penalty=100.0,
                                                     class_weights = 'auto',
                                                     validation_set=test_data_positive_ornot)

TypeError: create() got an unexpected keyword argument 'max_iterations'

In [33]:
feature_set_neutral_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_ornot']
model_neutral_ornot =graphlab.logistic_classifier.create(train_data_positive_ornot,
                                                     target='NeutralorNot',
                                                     features=feature_set_neutral_ornot,
                                                     max_iterations=100,
                                                     l1_penalty=0,
                                                     l2_penalty=6000.0,
                                                     class_weights = 'auto',
                                                     validation_set=test_data_positive_ornot)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9498
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 5
PROGRESS: Number of unpacked features : 267997
PROGRESS: Number of coefficients    : 267998
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 4        | 0.000053  | 0.585394     | 0.993578          | 0.637826            |
PROGRESS: | 2         | 6        | 1.000000  | 0.912611     | 0.997157          | 0.656087            |
PROGRESS: | 3         | 9        | 0.500000  | 1.354907     | 0.9964

In [317]:
feature_set_neutral_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_ornot']
model_neutral_ornot2 =graphlab.svm_classifier.create(train_data_positive_ornot,
                                                     target='NeutralorNot',
                                                     penalty = 5,
                                                     features=feature_set_neutral_ornot)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: SVM:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9008
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 5
PROGRESS: Number of unpacked features : 256438
PROGRESS: Number of coefficients    : 256439
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 3        | 0.000111  | 0.318217     | 0.913521          | 0.581633            |
PROGRESS: | 2    

In [331]:
coefficients = model_neutral_ornot['coefficients'] 

coefficients.print_rows(num_rows=1000, num_columns=4)

+-------------+------------------------+-------+--------------------+
|     name    |         index          | class |       value        |
+-------------+------------------------+-------+--------------------+
| (intercept) |          None          |   1   |  -0.416915693605   |
|    tfidf    |          sat.          |   1   | -0.000402498372747 |
|    tfidf    |           on           |   1   |  0.0013792924027   |
|    tfidf    |          hill          |   1   | -0.00168018345928  |
|    tfidf    |           ga           |   1   | -0.00276972369808  |
|    tfidf    |           to           |   1   | 0.000334328684012  |
|    tfidf    |           by           |   1   | -3.7489156981e-05  |
|    tfidf    |           i            |   1   | -0.00382513275871  |
|    tfidf    |         chapel         |   1   | -0.000885496207379 |
|    tfidf    |           my           |   1   | -0.00412605603479  |
|    tfidf    |          hit           |   1   |  -0.0013032830852  |
|    tfidf    |     

In [338]:
feature_set_neutral_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_ornot']
model_neutral_ornot3 =graphlab.sentiment_analysis.create(train_data_positive_ornot,
                                                     target='NeutralorNot',
                                                     features=feature_set_neutral_ornot)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8992
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 5
PROGRESS: Number of unpacked features : 255233
PROGRESS: Number of coefficients    : 255234
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 3        | 0.000111  | 0.356241     | 0.919373          | 0.606719            |
P

In [339]:
model_neutral_ornot3['classifier']

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 255234
Number of examples            : 8992
Number of classes             : 2
Number of feature columns     : 5
Number of unpacked features   : 255233

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.2

Training Summary
----------------
Solver                        : auto
Solver iterations             : 10
Solver status                 : TERMINATED: Iteration limit reached.
Training time (sec)           : 2.6177

Settings
--------
Log-likelihood                : 15.0696

Highest Positive Coefficients
-----------------------------
3gram features[someon see jurass]: 2.7662
3gram features[be po all]     : 2.484
3gram features[imaaa be po]   : 2.484
3gram features[still go to]   : 2.3836
3gram features[you still go]  : 2.3836

Lowest Negative Coefficients
----------------------------
1gram features[wohooo]        : -1.9288
2gram features[w

##Creating the Arbitrer classifier

In [1]:
#feature_set_pos_neg_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot','vectors_pos_neg','vectors_pos_neutral','vectors_neutral_neg','vectors_pos_ornot']
feature_set_pos_neg_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot','vectors_pos_neg','vectors_pos_neutral','vectors_neutral_neg','vectors_pos_ornot','vectors_doc2vec_tweetsonly_dm']
#feature_set_pos_neg_neutral = ['tfidf','1gram features','2gram features','3gram features']
#feature_set_pos_neg_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_doc2vec_tweetsonly_dm']
model_pos_neg_neutral =graphlab.classifier.create(train_data_positive_ornot,
                                                     target='Sentiment',
                                                     features=feature_set_pos_neg_neutral)

NameError: name 'graphlab' is not defined

In [2]:
import graphlab
test_tweets = graphlab.SFrame('local_test_sms_preprocessed.tsv')
#test_tweets['1gram features'] = gl.text_analytics.count_ngrams(test_tweets['Tweet'], 1)
#test_tweets['2gram features'] = gl.text_analytics.count_ngrams(test_tweets['Tweet'], 2)
#test_tweets['3gram features'] = gl.text_analytics.count_ngrams(test_tweets['Tweet'], 3)
test_tweets.head()

This non-commercial license of GraphLab Create for academic use is assigned to oaabde01@louisville.edu and will expire on September 22, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\OMARAB~1\AppData\Local\Temp\graphlab_server_1482636821.log.0


Finished parsing file C:\Users\OmarAbdelwahab\local_test_sms_preprocessed.tsv

Parsing completed. Parsed 100 lines in 0.036491 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\OmarAbdelwahab\local_test_sms_preprocessed.tsv

Parsing completed. Parsed 2069 lines in 0.018499 secs.

ID,Sentiment,Tweet
10936,neutral,ye i am go from schoolhave class till $NUM can ...
11051,neutral,can u tape the match forme? i\u$NUMl rush over ...
10966,neutral,too mani peopl at my housmy rel are here a po ...
11211,negative,yea i have spoken to himliao. inde he is ne over ...
11350,positive,haha... i want to see. emacdonald here cheaper. ...
10539,neutral,i\u$NUMm go down nowliao\u$NUMc with my ...
10544,neutral,ya\u$NUMc ok for me...erm can let me know the ...
11463,negative,he told u i\u$NUMm considliao mah. i duno\u$NU ...
11062,positive,will you be po on sundaynight? i just met ...
11035,neutral,tom\u$NUMc u think it isa relative\u$NUM hous or ...


In [80]:
dt = DeepTextAnalyzer(pos_neg_model_w2v)
test_tweets['vectors_pos_neg'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(pos_neutral_model_w2v)
test_tweets['vectors_pos_neutral'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(neutral_neg_model_w2v)
test_tweets['vectors_neutral_neg'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
#for the one label classifiers
dt = DeepTextAnalyzer(positive_nonpositive_model_w2v)
test_tweets['vectors_pos_ornot'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(negative_nonnegative_model_w2v)
test_tweets['vectors_neg_ornot'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))
dt = DeepTextAnalyzer(neutral_nonneutral_model_w2v)
test_tweets['vectors_neutral_ornot'] = test_tweets['Tweet'].apply(lambda p: dt.txt2avg_vector(p, is_html=False))

In [81]:
test_tweets['word_count'] = graphlab.text_analytics.count_words(test_tweets['Tweet'])
tfidf = graphlab.text_analytics.tf_idf(test_tweets['word_count'])
test_tweets['tfidf'] = tfidf

In [82]:
test_tweets['pos_neg'] = model_pos_neg_svm.predict(test_tweets)

In [83]:
test_tweets.head()

ID,Sentiment,Tweet,1gram features,2gram features
10936,neutral,ye i am go from schoolhave class till $NUM can ...,"{'am': 1L, 'num': 2L,'at': 2L, 'have': 3L, ...","{'till num': 1L, 'mumat': 1L, 'school have': ..."
11051,neutral,can u tape the match forme? i\u$NUMl rush over ...,"{'me': 1L, 'rush': 1L,'for': 1L, 'i': 1L, ...","{'tape the': 1L, 'forme': 1L, 'straight ..."
10966,neutral,too mani peopl at my housmy rel are here a po ...,"{'are': 1L, 'a': 1L,'china': 1L, 'hous': 1L, ...","{'a po': 1L, 'besid the':1L, 'are here': 1L, ..."
11211,negative,yea i have spoken to himliao. inde he is ne over ...,"{'dat': 1L, 'yea': 1L,'to': 1L, 'i': 1L, ...","{'liao inde': 1L, 'himliao': 1L, 'to him': 1L, ..."
11350,positive,haha... i want to see. emacdonald here cheaper. ...,"{'e': 1L, 'i': 1L,'haha': 1L, 'here': 1L, ...","{'to see': 1L, 'haha i':1L, 'want to': 1L, 'yum ..."
10539,neutral,i\u$NUMm go down nowliao\u$NUMc with my ...,"{'is': 2L, 'some': 1L,'it': 1L, 'one': 1L, ...","{'with my': 1L, 'numcwith': 1L, 'go around': ..."
10544,neutral,ya\u$NUMc ok for me...erm can let me know the ...,"{'btw': 1L, 'is': 1L,'times': 1L, 'for': 2L, ...","{'know the': 1L, 'numcpay': 1L, 'numc per': ..."
11463,negative,he told u i\u$NUMm considliao mah. i duno\u$NU ...,"{'shop': 2L, 'right': 1L,'dont': 1L, 'rest': 1L, ...","{'out on': 1L, 's n': 1L,'i dont': 1L, 'at num': ..."
11062,positive,will you be po on sundaynight? i just met ...,"{'play': 1L, 'just': 1L,'it': 1L, 'chit': 1L, ...","{'gather at': 1L, 'wesuggest': 1L, 'suggest ..."
11035,neutral,tom\u$NUMc u think it isa relative\u$NUM hous or ...,"{'a': 1L, 'hous': 1L,'is': 1L, 'numc': 1L, ...","{'tom u': 1L, 'relativeu': 1L, 'a relative': ..."


In [84]:
test_tweets['pos_neutral'] = model_pos_neutralt.predict(test_tweets)

In [85]:
test_tweets['neg_neutral'] = model_neutral_neg.predict(test_tweets)

In [86]:
test_tweets['negative_ornot'] = model_negative_ornot_svm.predict(test_tweets)

In [93]:
test_tweets.print_rows(num_rows=1400, num_columns=20)

+--------------------+-----------+-------------------------------+
|      TweetID       | Sentiment |             Tweet             |
+--------------------+-----------+-------------------------------+
| 260097528899452000 |  neutral  | won the match getin . plus... |
| 263791921753882000 |  neutral  | some area of new england c... |
| 260486470828171000 |  neutral  | tina fey amp; ami poehler ... |
| 262968617233162000 |  positive | lunch from my new lil spot... |
| 263790847424880000 |  positive | snc halloween pr. pumped. ... |
| 264255236435243000 |  negative | AT USER i m sorry, i heart... |
| 264221473558917000 |  neutral  | manchest unit will tri to ... |
| 264091690632105000 |  neutral  | go to a bull game with aal... |
| 263929564907069000 |  neutral  | ani toon fan with a spare ... |
| 263759328782204000 |  positive | loui inspir outfit on mond... |
| 259546192722161000 |  neutral  | go to bed now...ros parad ... |
| 264249030572392000 |  neutral  | AT USER oh caus my friend .

In [87]:
test_tweets['pos_neg_neutral'] = model_pos_neg_neutral.predict(test_tweets)

##End of preprocessing

In [65]:
test_tweets2['word_count'] = graphlab.text_analytics.count_words(test_tweets2['Tweet'])
tfidf = graphlab.text_analytics.tf_idf(test_tweets2['word_count'])
test_tweets2['tfidf'] = tfidf

In [77]:
#test_tweets2['neg_ornot'] = model_negative_ornot.predict(test_tweets2['Tweet','vectors_neg_ornot'])
test_tweets2.print_rows(num_rows=1000, num_columns=15)

NameError: name 'test_tweets2' is not defined

In [72]:
import pickle

In [73]:
f = open('C:\\Users\OmarAbdelwahab\\Documents\\RESEARCH\\SEMEVAL2016\\TwitterSentimentAnalysisTask\\Train-Trial-Data\\2016Data\\Train+dev-2016\\Needed\\Train\\clf.pickle', 'rb')
classifier_word2vec = pickle.load(f)
f.close()

In [131]:
feature_set_pos_neg_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot','vectors_pos_neg']
svm_model = graphlab.classifier.create(train_data_pos_neg, target='Sentiment',
                                 features=feature_set_pos_neg_neutral)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5155
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 6
PROGRESS: Number of unpacked features : 157648
PROGRESS: Number of coefficients    : 157649
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [132]:
svm_model.evaluate(test_data_pos_neg)

{'accuracy': 0.8344774980930587, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   negative   |     positive    |  170  |
 |   negative   |     negative    |  143  |
 |   positive   |     negative    |   47  |
 |   positive   |     positive    |  951  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.8975932043416706, 'precision': 0.848349687778769, 'recall': 0.9529058116232465}

In [74]:
#feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot','vectors_doc2vec_tweetsonly_dm']
#feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_neg_ornot']
feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features']
#feature_set_negative_ornot = ['tfidf','1gram features','2gram features','3gram features','vectors_doc2vec_tweetsonly_dm']
model_negative_ornot_svm =graphlab.classifier.create(tweets,
                                                     target='NegativeorNot',
                                                     features=feature_set_negative_ornot,
                                                    
                                                     )

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 11188
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 305069
PROGRESS: Number of coefficients    : 305070
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +---

In [75]:
#feature_set_pos_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neg','vectors_pos_neutral','vectors_pos_ornot','vectors_neg_ornot','vectors_doc2vec_tweetsonly_dm']
#feature_set_pos_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neg','vectors_pos_neutral','vectors_pos_ornot','vectors_neg_ornot']
feature_set_pos_neg = ['tfidf','1gram features','2gram features','3gram features']
#feature_set_pos_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_doc2vec_tweetsonly_dm']
model_pos_neg_svm =graphlab.classifier.create(tweets_pos_neg,
                                                     target='Sentiment',
                                                     features=feature_set_pos_neg)


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 6411
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 188877
PROGRESS: Number of coefficients    : 188878
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [145]:
feature_set_pos_neutral = ['tfidf','1gram features','2gram features','3gram features','vectors_pos_neutral','vectors_pos_ornot','vectors_neutral_ornot']
model_pos_neutral_svm =graphlab.classifier.create(tweets_pos_neutral,
                                                     target='Sentiment',
                                                     features=feature_set_pos_neutral)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9658
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 7
PROGRESS: Number of unpacked features : 268905
PROGRESS: Number of coefficients    : 268906
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

In [152]:
feature_set_neutral_neg = ['tfidf','1gram features','2gram features','3gram features','vectors_neutral_neg','vectors_neg_ornot','vectors_neutral_ornot']
model_neutral_neg_lr_svm =graphlab.classifier.create(tweets_neutral_neg,
                                                     target='Sentiment',
                                                     features=feature_set_neutral_neg)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 6395
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 7
PROGRESS: Number of unpacked features : 196209
PROGRESS: Number of coefficients    : 196210
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +----

##Aggregation

In [88]:
import math
final_sentimentlist = []
for i in test_tweets:
    pos=0
    neg=0
    neutral=0
    final_sentiment = 'unknown'
    if i['pos_neg'] == 'positive':
        pos = math.fsum([pos,1])
    
    if i['pos_neg'] == 'negative':
        neg= math.fsum([neg,1])
    if i['pos_neutral'] == 'positive':
        pos = math.fsum([pos,1])
    if i['pos_neutral'] == 'neutral':
        neutral = math.fsum([neutral,1])
    if i['neg_neutral'] == 'negative':
        neg = math.fsum([neg,1])
    if i['neg_neutral'] == 'neutral':
        neutral = math.fsum([neutral,1])
    if i['negative_ornot']==1:
        final_sentiment = 'negative'
    if(final_sentiment!='negative' and pos>neg and pos>neutral):
        final_sentiment='positive'
    if(final_sentiment!='negative' and neg>pos and neg>neutral):
        final_sentiment='negative'
    if(final_sentiment!='negative' and neutral>pos and neutral>neg):
        final_sentiment='neutral'
    if(final_sentiment == 'negative'):
        i['end_sentiment'] = 'negative'
    if(final_sentiment == 'positive'):
        i['end_sentiment'] = 'positive'
    if(final_sentiment == 'neutral'):
        i['end_sentiment'] = 'neutral'
    if(i['negative_ornot']==0 and pos==neg and pos==neutral and neutral==neg):
        final_sentiment = i['pos_neg_neutral']
    if(final_sentiment=='unknown'):
        final_sentiment = i['pos_neg_neutral']
    print final_sentiment
    final_sentimentlist.append(final_sentiment)
    


neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral

In [89]:
test_tweets['last_sentiment'] = final_sentimentlist

In [3]:
test_tweets.head()

ID,Sentiment,Tweet
10936,neutral,ye i am go from schoolhave class till $NUM can ...
11051,neutral,can u tape the match forme? i\u$NUMl rush over ...
10966,neutral,too mani peopl at my housmy rel are here a po ...
11211,negative,yea i have spoken to himliao. inde he is ne over ...
11350,positive,haha... i want to see. emacdonald here cheaper. ...
10539,neutral,i\u$NUMm go down nowliao\u$NUMc with my ...
10544,neutral,ya\u$NUMc ok for me...erm can let me know the ...
11463,negative,he told u i\u$NUMm considliao mah. i duno\u$NU ...
11062,positive,will you be po on sundaynight? i just met ...
11035,neutral,tom\u$NUMc u think it isa relative\u$NUM hous or ...


In [59]:
test_tweets.print_rows(num_rows=50, num_columns=20)

+----+-----------+-------------------------------+
| ID | Sentiment |             Tweet             |
+----+-----------+-------------------------------+
| 1  |  UNKNOWN  | i m done write code for th... |
| 2  |  UNKNOWN  | seven penni stock on the m... |
| 3  |  UNKNOWN  | dec $NUMst $NUM will be kn... |
| 4  |  UNKNOWN  | yar he quit po but aft man... |
| 5  |  UNKNOWN  | yeah we have thin lizzi he... |
| 6  |  UNKNOWN  | mt AT USER syria\u$NUMc de... |
| 7  |  UNKNOWN  | AT USER ne my life, becaus... |
| 8  |  UNKNOWN  | ard $NUM\u$NUMc $NUM time ... |
| 9  |  UNKNOWN  | po for fri\u$NUMcnot so i ... |
| 10 |  UNKNOWN  | just got the ic updat from... |
| 11 |  UNKNOWN  | check out our ladi peac at... |
| 12 |  UNKNOWN  | hmm... mon afternoon i got... |
| 13 |  UNKNOWN  | just thought about the fac... |
| 14 |  UNKNOWN  | bedtime. go to dayton at $... |
| 15 |  UNKNOWN  | we eat at science\u$NUMc u... |
| 16 |  UNKNOWN  | sign of the time for onc p... |
| 17 |  UNKNOWN  | hello from t

In [91]:
del test_tweets['Sentiment']
del test_tweets['Tweet']
del test_tweets['1gram features']
del test_tweets['2gram features']
del test_tweets['3gram features']
del test_tweets['vectors_pos_neg']
del test_tweets['vectors_pos_neutral']
del test_tweets['vectors_pos_ornot']
del test_tweets['vectors_neg_ornot']
del test_tweets['vectors_neutral_ornot']
del test_tweets['word_count']
del test_tweets['tfidf']
del test_tweets['pos_neg']
del test_tweets['pos_neutral']
del test_tweets['neg_neutral']
del test_tweets['negative_ornot']
del test_tweets['pos_neg_neutral']
del test_tweets['vectors_neutral_neg']

In [4]:
del test_tweets['Sentiment']
del test_tweets['ID']
test_tweets.head()

Tweet
ye i am go from schoolhave class till $NUM can ...
can u tape the match forme? i\u$NUMl rush over ...
too mani peopl at my housmy rel are here a po ...
yea i have spoken to himliao. inde he is ne over ...
haha... i want to see. emacdonald here cheaper. ...
i\u$NUMm go down nowliao\u$NUMc with my ...
ya\u$NUMc ok for me...erm can let me know the ...
he told u i\u$NUMm considliao mah. i duno\u$NU ...
will you be po on sundaynight? i just met ...
tom\u$NUMc u think it isa relative\u$NUM hous or ...


In [93]:
test_tweets.save('C:\\Users\\OmarAbdelwahab\\Documents\\RESEARCH\\SEMEVAL2016\\TwitterSentimentAnalysisTask\\Train-Trial-Data\\2016Data\\Train+dev-2016\\Needed\\Train\\local_results_worst.csv',format='csv')

In [9]:
test_tweets.save('testing.csv',format='csv')

## SEMEVAL 2017 preprocessing

In [3]:
tweets

TweetID,Sentiment,Tweet
264183816548130000,positive,ga by my hous hit$NUM.$NUM i m go to ...
264249301910310000,negative,iranian gener say israels iron dome can t deal ...
264105751826538000,positive,with j davlar $NUMth.main rival are team ...
264094586689953000,negative,"talk about act s amp;amp; sat s, decid i ..."
254941790757601000,negative,"they may have a superbowlin dalla, but dalla a ..."
264169034155696000,neutral,"im bring the ne load ofcandi tomorrow, i just ..."
263192091700654000,neutral,"appl software, retailchief out in overhaul: ..."
263398998675693000,positive,AT USER AT USER AT USER ijust watch it sridevi s ...
260200142420992000,neutral,livewir nadal confirm formexican open in febru ...
264087629237202000,positive,AT USER i didnt want tojust pop up... but ye ...


In [27]:
pos_tweets = tweets[tweets['Sentiment'] == 'positive']

In [28]:
neg_tweets = tweets[tweets['Sentiment'] == 'negative']

In [53]:
del pos_tweets['ID']
del pos_tweets['Sentiment']
del pos_tweets['Tweet']
del pos_tweets['clean_tweet']

In [54]:
pos_tweets

cut_tweet
Microsoft I may notprefer your gaming br ...
Just ordered my 1st evertablet Microsoft Surface ...
Sunday morning quiet dayso time to welcome in ...
Innovation for jobs isjust around the corner ...
Vote for AIESEC to becomethe 10th Global non ...
Top 5 most searched forBacktoSchool topics the ...
truckersquigz MicrosoftMISpeedway nationwide88 ...
ScottArbeit GabeAulMicrosoft isntall the ...
taehongmin1 We have anIOT workshop by Micro ...
ProfessorF gilwuvsyouMicrosoft LivioDeLaCruz ...


In [52]:
del neg_tweets['ID']
del neg_tweets['Sentiment']
del neg_tweets['Tweet']
del neg_tweets['clean_tweet']

In [51]:
neg_tweets

ID,Sentiment,Tweet,clean_tweet
628949369883000832,negative,dear @Microsoft thenewOoffice for Mac is ...,dear Microsoft thenewOoffice for Mac is ...
628976607420645377,negative,@Microsoft how about youmake a system that ...,Microsoft how about youmake a system that do ...
629345637155360768,negative,@MikeWolf1980 @MicrosoftI will be downgrading ...,MikeWolf1980 Microsoft Iwill be downgrading and ...
629394528336637953,negative,@Microsoft 2nd computerwith same error!!! ...,Microsoft 2nd computerwith same error ...
629797991826722816,negative,"After attempting areinstall, it still ...",After attempting areinstall it still br ...
630542330827771904,negative,Did @Microsoft breakWindows 10? Was working ...,Did Microsoft breakWindows 10 Was working ...
630636736746422272,negative,@MSAU @Microsoft spentover 40 mins &amp; gave ...,MSAU Microsoft spent over40 mins amp gave up coz ...
631104156187627520,negative,"For the 1st time @Skypehas a ""High Startup ...",For the 1st time Skypehas a High Startup im ...
631223085476261890,negative,#teens @BillGates 1stcompany failed misera ...,teens BillGates 1stcompany failed miserably ...
631543121407442946,negative,@Microsoft support for365 has been terrible ...,Microsoft support for 365has been terrible On the ...


In [55]:
pos_tweets.save('pos_tweets.csv',format='csv')

In [56]:
neg_tweets.save('neg_tweets.csv',format='csv')

In [57]:
def cut_text(tex):
    counter = 0
    word_counter = 0
    for c in tex:
        counter=counter+1
        if c==' ':
            word_counter=word_counter + 1
        if word_counter == 56:
            break
    return tex[:counter]

In [50]:
pos_tweets['cut_tweet'] = pos_tweets['clean_tweet'].apply(lambda x: cut_text(x))
neg_tweets['cut_tweet'] = neg_tweets['clean_tweet'].apply(lambda x: cut_text(x))

In [34]:
neg_tweets['clean_tweet'] = neg_tweets['Tweet'].apply(lambda x: cut_text(x))

In [35]:
del pos_tweets['Tweet']

In [36]:
del neg_tweets['Tweet']

In [37]:
pos_tweets[2]

{'clean_tweet': 'AT USER AT USER AT USER i just watch it sridevi s comeback.... u rememb her from the $NUMs?? sun morn on nta ;)'}

In [58]:
pos_tweets = graphlab.SFrame('PositiveTweets-A-NA.tsv')
neg_tweets = graphlab.SFrame('NegativeTweets-A-NA.tsv')

Finished parsing file C:\Users\OmarAbdelwahab\PositiveTweets-A-NA.tsv

Parsing completed. Parsed 100 lines in 0.025501 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\OmarAbdelwahab\PositiveTweets-A-NA.tsv

Parsing completed. Parsed 2814 lines in 0.021509 secs.

Finished parsing file C:\Users\OmarAbdelwahab\NegativeTweets-A-NA.tsv

Parsing completed. Parsed 100 lines in 0.020999 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\OmarAbdelwahab\NegativeTweets-A-NA.tsv

Parsing completed. Parsed 775 lines in 0.018001 secs.

In [59]:
pos_tweets

ID,Sentiment,Tweet
629226490152914944,positive,"Microsoft, I may notprefer your gaming br ..."
629650766580609026,positive,Just ordered my 1st evertablet; @Microsoft ...
630159517058142208,positive,"Sunday morning, quiet dayso time to welcome in ..."
630818265799921664,positive,Innovation for jobs isjust around the corner - ...
631368262979297281,positive,#Vote for @AIESEC tobecome the 10th Global ...
631521079245307904,positive,Top 5 most searched forBack-to-School topics -- ...
631696872323850240,positive,@trucker_squigz@Microsoft @MISpeedway ...
631842974268305408,positive,@ScottArbeit @GabeAul@Microsoft isntall the ...
631843393971204097,positive,@taehongmin1 We have anIOT workshop by ...
633628599271190528,positive,@ProfessorF @gilwuvsyou@Microsoft @LivioDeLa ...


In [47]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation)

In [60]:
pos_tweets['clean_tweet'] = pos_tweets['Tweet'].apply(lambda x: remove_punctuation(x))

In [61]:
neg_tweets['clean_tweet'] = neg_tweets['Tweet'].apply(lambda x: remove_punctuation(x))

In [62]:
pos_tweets['cut_tweet'] = pos_tweets['clean_tweet'].apply(lambda x: cut_text(x))
neg_tweets['cut_tweet'] = neg_tweets['clean_tweet'].apply(lambda x: cut_text(x))

In [64]:
tweets = pos_tweets

In [65]:
tweets = tweets.append(neg_tweets)

In [66]:
tweets.save('tweets17.csv',format='csv')

In [68]:
del tweets['ID']
del tweets['Sentiment']
del tweets['Tweet']
del tweets['clean_tweet']

In [69]:
tweets.save('tweets17_raw.csv',format='csv')